# Data Processing 

This code extracts the date, ticker and text from the json file in the NASDAQ news that can be obtained via the Blockchain Research Center (https://blockchain-research-center.com), and creates a csv file with "Date", "Ticker", "Text", "Text_unprocessed","Word Count", "Permno".

Further the code adjusts the date format such that it can be used to fetch data from the WRDS database.
The text is processed to reduce noise with the help of the functions from Frankel, Jennings and Lee (2021)

In [ ]:
import csv
import sys
import pandas as pd
import json
import string
from nltk.stem.porter import *
import wrds
from datetime import datetime
stemmer = PorterStemmer()
exclude = set(string.punctuation)

#connect to WRDS with your own account
db = wrds.Connection(wrds_username="your username")

#To max out field limit
csv.field_size_limit(sys.maxsize)

json_file_path = "NASDAQ_News.json"
output_file_path = "output_path"
output_file_name = "process_data.csv"

#### FUNCTIONS TO CLEAN PHRASES AND WORDS #### Frankel, Jennings and Lee (2021)

#stopwords = ['a','able','across','after','also','am','among','an','and','any','are','as','at','be','because','been','but','by','can','could','dear','did','do','does','either','else','ever','every','for','from','get','got','had','has','have','he','her','hers','him','his','how','however','i','if','in','into','is','it','its','just','let','like','likely','me','my','of','off','often','on','only','or','other','our','own','rather','said','say','says','she','should','since','so','some','than','that','the','their','them','then','there','these','they','this','tis','to','too','twas','us','wants','was','we','were','what','when','where','which','while','who','whom','why','will','with','would','yet','you','your']
stopwords = []

stopwords_dict={}
for stopword in stopwords:
    stopwords_dict[stopword]=0

def fix_phrases(section):
    section = re.sub('(\d+)\.(\d+)','\g<1>\g<2>',section) # Remove periods from numbers -- 4.55 --> 455
    section = section.replace(".com", "com")
    section = section.replace("-", " ")
    section = section.replace('. .', '.')
    section = section.replace('.', 'XXYYZZ1')
    section = ''.join(ch for ch in section if ch not in exclude) #Delete all punctuation except periods
    section = section.replace('XXYYZZ1', '.')
    section = section.lower()
    section = re.sub(' +',' ',section) #Remove multiple spaces
    if section == '.': section = ''
    return section

def fixword(word, portstem = True):
        word = word.replace('\n',' ')
        if re.search('[0-9]',word) != None:
            word = '00NUMBER00' # Replace numbers with 000NUMBER000
        try:
            test = stopwords_dict[word]
            word = '_' # Replace stop words with _
        except Exception:
            donothing = 1
        #Variable if stemming or not
        if portstem:
            try:
                word = stemmer.stem(word)  # Stemp words
            except Exception:
                word = ''
        return word

def split_years(dt):
    dt["Year"] = dt["Date"].dt.year
    return [dt[dt["Year"] == y] for y in dt["Year"].unique()]

def split_weeks(dt):
    dt["Week"] = dt["Date"].dt.isocalendar().week
    return [dt[dt["Week"] == y] for y in dt["Week"].unique()]

with open(json_file_path, 'r') as json_file:

    data_fill = []

    count_ticker1_only = 0
    count_permno_match = 0

    for i, line in enumerate(json_file):

        print(i)

        data = json.loads(line)

        try:
            date = data["article_time"]["$date"]
        except KeyError:
            continue

        try:
            ticker = data["symbols"]
        except KeyError:
            continue

        try:
            text = data["article_content"]
        except KeyError:
            continue

        if len(ticker.split(sep=",")) == 1 and ticker != "":

            count_ticker1_only += 1

            # Change date to proper format without T and Z
            date = list(date)
            date = [w.replace("T", " ") for w in date]
            date = [w.replace("Z", " ") for w in date]
            date = "".join(date)
            date_str = pd.to_datetime(date.split(" ")[0])
            date = pd.to_datetime(date)
            date_str = date_str.strftime("%m/%d/%Y")

            #Keep text unprocessed 
            text_unprocessed = text

            # Pre-processing text to reduce noise. Part of the code is taken from Frankel, Jennings and Lee (2021)
            text = fix_phrases(text)
            sentences = text.split('.')

            count_words = 0

            for v, sentence in enumerate(sentences):

                sentences[v] = sentences[v].replace(".", "").strip()

                allwords = sentences[v].split(" ")

                for w, word in enumerate(allwords):
                    allwords[w] = fixword(allwords[w], portstem=False)
                    if allwords[w].strip() != '.' and allwords[w].strip() != '':
                        count_words += 1

                sentences[v] = " ".join(allwords)

            text = ".".join(sentences)
            text = text.replace(".", ". ")

            #Get permno of company for fetching data from wrds. If no PERMNO match then observation will not be used in sentiment analysis.

            permno = db.raw_sql("""select permno
                                            from crsp.dse
                                            where TICKER = '{}'
                                            and date  <= '{}'""".format(ticker, date_str))

            try:
                permno["permno"].iloc[-1]
                count_permno_match += 1
            except IndexError:
                continue

            data_fill.append([date, ticker, text, text_unprocessed ,count_words, permno["permno"].iloc[-1]])


    data = pd.DataFrame(data_fill, columns = ["Date", "Ticker", "Text", "Text_unprocessed","Word Count", "Permno"])

    data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")
    data.sort_values(by="Date", inplace=True)
    data.drop_duplicates(inplace=True)
    dropped_duplicates = len(data)

    data_splt_years = split_years(data)

    sorteddflist = sorted(data_splt_years, key=lambda x: x["Year"].min(axis=0))

    sorted_final = []

    for year in sorteddflist:
        year = split_weeks(year)
        year_sorted = sorted(year, key=lambda x: x["Week"].min(axis=0))
        sorted_final.append(pd.concat(year_sorted))

    df = pd.concat(sorted_final)

    df["Quarter"] = df["Date"].dt.quarter
    df = df[["Date", "Ticker", "Text", "Text_unprocessed","Word Count", "Permno"]]

    json_file.close()

    print("Observations with only 1 associated stock ticker symbol: {}".format(count_ticker1_only))
    print("Observations with a CRSP PERMNO match: {}".format(count_permno_match))
    print("Observations after all duplicates are dropped: {}".format(dropped_duplicates))

with open(output_file_path + "data_count_initial.csv", "w") as f:
  writer = csv.writer(f)
  writer.writerow([count_ticker1_only, count_permno_match, dropped_duplicates])

with open(output_file_path + output_file_name, "w") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Date", "Ticker", "Text", "Text_unprocessed","Word Count", "Permno"])
    for index, row in df.iterrows():
        writer.writerow(row)


Streaming output truncated to the last 5000 lines.
1405535
1405536
1405537
1405538
1405539
1405540
1405541
1405542
1405543
1405544
1405545
1405546
1405547
1405548
1405549
1405550
1405551
1405552
1405553
1405554
1405555
1405556
1405557
1405558
1405559
1405560
1405561
1405562
1405563
1405564
1405565
1405566
1405567
1405568
1405569
1405570
1405571
1405572
1405573
1405574
1405575
1405576
1405577
1405578
1405579
1405580
1405581
1405582
1405583
1405584
1405585
1405586
1405587
1405588
1405589
1405590
1405591
1405592
1405593
1405594
1405595
1405596
1405597
1405598
1405599
1405600
1405601
1405602
1405603
1405604
1405605
1405606
1405607
1405608
1405609
1405610
1405611
1405612
1405613
1405614
1405615
1405616
1405617
1405618
1405619
1405620
1405621
1405622
1405623
1405624
1405625
1405626
1405627
1405628
1405629
1405630
1405631
1405632
1405633
1405634
1405635
1405636
1405637
1405638
1405639
1405640
1405641
1405642
1405643
1405644
1405645
1405646
1405647
1405648
1405649
1405650
1405651
1405652
14056

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Observations with only 1 associated stock ticker symbol: 324679
Observations with a CRSP PERMNO match: 314826
Observations after all duplicates are dropped: 311683
